### Imports

In [ ]:
import mysql.connector, json, os, requests, re
from bs4 import BeautifulSoup
conn = mysql.connector.connect(
    user=os.environ['MYSQL_DB_USER'], 
    password=os.environ['MYSQL_DB_PASSWORD'], 
    host='127.0.0.1', 
    database='etymology_explorer_staging')
cursor = conn.cursor()

In [ ]:
all_pos = ['adfix', 'adjective', 'adnoun', 'adverb', 'article', 'auxiliary verb', 'cardinal number', 'collective numeral',
           'conjunction', 'coverb', 'demonstrative determiner', 'demonstrative pronoun', 'determinative', 'determiner',
           'gerund', 'indefinite pronoun', 'infinitive', 'interjection', 'interrogative pronoun', 'intransitive verb',
           'noun', 'number', 'numeral', 'ordinal', 'ordinal number', 'part of speech', 'participle', 'particle',
           'personal pronoun', 'phrasal preposition', 'possessive adjective', 'possessive determiner', 'possessive pronoun',
           'postposition', 'preposition', 'preverb', 'pronoun', 'quasi-adjective', 'reciprocal pronoun', 'reflexive pronoun',
           'relative pronoun', 'speech disfluency', 'substantive', 'transitive', 'transitive verb', 'verb', 'verbal noun''suffix', 
           'prefix', 'infix', 'root']

### Get urls

Ok so I want to have the data staged somewhere so that I can check it before I update everything else. I could move it into a dataframe. I already have one that stores a ton of information. The definitions could be one big array, or it could be it's own csv with definitions as a separate list. I think I'm going to do that first. It is more flexible. And I can do that for the pronunciations. It will be the same as the other DF. I think I could also just have it append all the results to a file
- word, language, etymology
- word, language, pronunciation
- word, language, pos, definition

Cat is a good test for multiple etymologies

In [ ]:
import pandas as pd

# Get the list of word-language pairs
df = pd.read_csv('~/etymology_files/ety_master.csv', 
                 usecols = ['word', 'language'], 
                 converters={'word' : str, 'language': str}
                )

# Set all non-reconstructions to be 'None'
normal_language_rows = [not language.startswith('Proto') for language in df['language'].tolist()] #bools
df.loc[normal_language_rows, 'language'] = None
df = df.drop_duplicates()
url_terms = [row[0] if row[1] is None else 'Reconstruction:'+row[1]+'/'+row[0] for row in df.values]
urls = ['https://en.wiktionary.org/api/rest_v1/page/html/' + term for term in url_terms]

Now try from MYSQL

In [ ]:
self.user = os.environ['MYSQL_DB_USER']
self.password = os.environ['MYSQL_DB_PASSWORD']
self.host = '127.0.0.1'
self.database = 'etymology_explorer_staging'
self.conn = mysql.connector.connect(user=self.user, password=self.password, host=self.host, database=self.database)
self.cursor = self.conn.cursor()

In [ ]:
cursor.execute('SELECT word, language_name \
                    FROM etymologies e, languages l\
                    WHERE e.language_code = l.language_code AND _id NOT IN (SELECT etymology_id FROM entry_connections) \
                    limit 10')
wls = [[w.decode('utf-8','replace').strip(), l.decode()] for w, l in cursor.fetchall()]; wls
url_terms = [row[0] if not row[1].startswith('Proto') else 'Reconstruction:'+row[1]+'/'+row[0] for row in wls]; url_terms
# for url in set(url_terms):
#     print(url)

In [ ]:
df = pd.read_csv('~/etymology_files/ety_master.csv',
                 usecols = ['word', 'language'],
                 converters={'word' : str, 'language': str})

        # Set all non-reconstructions to be 'None'
        normal_language_rows = [not language.startswith('Proto') for language in df['language'].tolist()] #bools
        df.loc[normal_language_rows, 'language'] = None
        df = df.drop_duplicates()
        url_terms = [row[0] if row[1] is None else 'Reconstruction:'+row[1]+'/'+row[0] for row in df.values]
        urls = ['https://en.wiktionary.org/api/rest_v1/page/html/' + term for term in url_terms[:5] + ['cat']]

### Scrape Wiktionary Page

In [ ]:
def getDefsFromPOS(ety_pronunc_pos_node):
    keep_def_tags = ('i', 'b', 'a', 'span', None)
    node_data = []
    for li in list(ety_pronunc_pos_node.parent.find('ol').children): # get defs from ordered list
        if li.name != 'li': continue # This is a newline tag

        if li.find('ol'): #Ordered list means the sub items are the definition
            for sub_li in list(li.find('ol').children):
                if sub_li.name != 'li': continue # Skip newline tags

                for child in sub_li.children:
                    if child.name not in keep_def_tags: child.clear() #Get rid of quotes and subitems

                node_data.append(sub_li.text.strip())

        else: # otherwise grab the text

            for child in li.children:
                if child.name not in keep_def_tags: child.clear() #Get rid of quotes and subitems
            node_data.append(li.text.strip())
    return node_data
    

In [ ]:
word='self-referential'#'Reconstruction:Proto-Indo-European%2Fkerp-'; #word='犬' #ǵerh₂-
response = requests.get(f'https://en.wiktionary.org/api/rest_v1/page/html/{word}'); response
page = response.url.replace('https://en.wiktionary.org/api/rest_v1/page/html/', '')
page = re.sub('Reconstruction:[^\/]+?\/(.*)', r'\1', page) #Remove reconstruction text if necessary
soup = BeautifulSoup(response.content, 'html.parser')

page_data = {'term': page} # Variable to store the data

for lang_node in soup.find_all('h2'): # Go through each Language
    print(lang_node)
    language = lang_node.text.strip()
    language_entries = [{}]

    for ety_pronunc_pos_node in lang_node.parent.find_all('h3'): # Go through each ety,pronu, or pos
        print(ety_pronunc_pos_node)
        node_data = []
        node_class = re.sub('_\d+| \d+', '', ety_pronunc_pos_node.text).lower() #removed '_x' info
#         node_text = ety_pronunc_pos_node.text.lower()

        if node_class == 'etymology':
            #Only looking at the first <p> element for etymology text
            etymology_text = ety_pronunc_pos_node.parent.find('p').text.replace('\u200e', '')
            entry_data = {'etymology': etymology_text}
            
            for sub_ety_pos in ety_pronunc_pos_node.parent.find_all('h4'):
                if sub_ety_pos.text.lower() in all_pos:
                    entry_data[sub_ety_pos.text.lower()] = getDefsFromPOS(sub_ety_pos)

            if any(['etymology' in entry for entry in language_entries]): #If an etymology already exists add to new entry
                language_entries.append(entry_data)
            else:
                language_entries[0].update(entry_data)
                
        # Need to see if there are sub items of this etymology
        elif node_class == 'pronunciation':
            ipa_nodes = ety_pronunc_pos_node.parent.select('span.IPA') #dataquest.io/blog/web-scraping-tutorial-python/
            if ipa_nodes: #Only add pronunciation if there are ipa_nodes
                node_data = ipa_nodes[0].text
                language_entries[0]['pronunciation'] = node_data

        elif node_class in all_pos: # Here are the definitions
            language_entries[0][node_class] = getDefsFromPOS(ety_pronunc_pos_node)

        else: # Skip all other node_classes
            continue

        page_data[language] = language_entries #Add all the language entries to the language 
print (json.dumps(page_data, indent=4))

### Parsed response into SQL

- Languages (1,2,3,4)
    - Etymology (1,2,3,4)
    - Pronunciation (1,2,3,4)
    - POS (1,2,3,4)
        - Definitions (1,2,3,4

Need the scraper to determine the number of entries:
- Hand (English) has one entry with multiple POS and one etymology
- Cat (English) has 9 entries each with one etymology and multiple POS
- Are there any entries with multiple etymologies?

In [ ]:
def getNewKey(column, table):
    cursor.execute(f'SELECT max({column}) FROM {table}')
    max_entry_id = cursor.fetchone()[0]; 
    new_entry_id = max_entry_id + 1 if max_entry_id is not None else 0
    return new_entry_id

In [ ]:
def insert(table, **kwargs):
    columns = [str(k) for k,v in kwargs.items()]; columns
    values = [str(v) if type(v) != str else repr(v)  for k,v in kwargs.items()]; values
    col_text = '('+', '.join(columns)+')'
    val_text = '('+', '.join(values)+')'
    sql_statement = f'INSERT INTO {table}{col_text} VALUES {val_text}'
    print(sql_statement)
    cursor.execute(sql_statement)

In [ ]:
conn.close()

In [ ]:
conn.commit()

In [ ]:
text = """{'English': [{'etymology': 'From Middle English apprile, Aprill, re-Latinised from Middle English aueril, from Old French avrill, from Latin Aprīlis (“of the month of the goddess Venus”), perhaps based on Etruscan 𐌖𐌓𐌐𐌀 (Apru), from Ancient Greek Αφροδίτη (Aphrodítē, “Venus”).', 'pronunciation': '/ˈeɪ.pɹɪl/'}], 'Afrikaans': [{'noun': ['April']}], 'Cebuano': [{'etymology': 'From English April, from Middle English apprile, re-Latinized from aueril, from Old French avrill, from Latin Aprīlis (“of the month of the goddess Venus”), perhaps based on Etruscan 𐌖𐌓𐌐𐌀 (Apru), from Ancient Greek Αφροδίτη (Aphrodítē, “Venus”).'}], 'German': [{'pronunciation': '/aˈpʁɪl/', 'noun': ['April']}], 'Malay': [{'etymology': 'From English April, from Middle English apprile, from aueril, from Old French avrill, from Latin Aprīlis.', 'pronunciation': '[aprel]'}]}"""
table = 'entry_etymologies'
kwargs = {'etymology': text}
columns = [str(k) for k,v in kwargs.items()]; columns
values = [str(v) if type(v) != str else repr(v)  for k,v in kwargs.items()]; values
col_text = '('+', '.join(columns)+')'
val_text = '('+', '.join(values)+')'


cursor.execute('SET NAMES utf8mb4;')
# cursor.execute('SET CHARACTER SET utf8mb4;')
# cursor.execute('SET character_set_connection=utf8mb4;')

cursor.execute("INSERT INTO entry_etymologies(etymology) VALUES (%s)", [text.encode()])

In [ ]:
# Clear out the databases
cursor.execute('DELETE FROM entry_connections')
cursor.execute('DELETE FROM entry_pronunciations')
cursor.execute('DELETE FROM entry_etymologies')
cursor.execute('DELETE FROM entry_pos')
cursor.execute('DELETE FROM entry_definitions')
conn.commit()

In [ ]:
with open('/home/ubuntu/scrapy/wiktionary_scraper/output/test3.json', 'r') as results:
    for line in results:
        this_data = json.loads(line)
        if this_data['term'] == 'cat': break
        
# print('SCRAPED DATA')
# print(json.dumps(this_data, indent=4))
        
word = this_data['term']
del this_data['term']

for lang, entries in this_data.items():
#     print()
#     print('STORED DATA')
    language = lang
#     print(language)
#     word='asdfasdf'
    cursor.execute(f'SELECT _id FROM etymologies e, languages l WHERE word = "{word}" and language_name = "{language}" and e.language_code = l.language_code')
    ety_id_result = cursor.fetchone(); etymology
    if ety_id_result is not None: 
        ety_id = ety_id_result[0]
    else:
        ety_id = getNewKey('_id', 'etymologies') + 1
        cursor.execute(
                    f'INSERT INTO etymologies(_id, word, language_code) \
                        SELECT {ety_id}, "{word}", language_code FROM languages WHERE language_name = "{language}"')
    
    for entry in entries:
        # Get a new entry key, make the entry connection
        new_entry_id = getNewKey('entry_id', 'entry_connections'); print("new_entry_id:", new_entry_id)
        insert('entry_connections', etymology_id=ety_id, entry_id=new_entry_id)
        
        for node_key, node_value in entry.items():
            if node_key == 'pronunciation':
#                 print('node k,v:', node_key, node_value)
                insert('entry_pronunciations', pronunciation=node_value, entry_id=new_entry_id)
    
            elif node_key == 'etymology':
                insert('entry_etymologies', etymology=node_value, entry_id=new_entry_id)
                
            elif node_key in all_pos:
                new_pos_key = getNewKey('pos_id', 'entry_pos'); print("new_pos_key:", new_pos_key)
                insert('entry_pos', pos=node_key, pos_id=new_pos_key, entry_id=new_entry_id)
                
                for definition in node_value:
#                     print('definition:', definition, new_pos_key)
                    insert('entry_definitions', definition=definition, pos_id=new_pos_key)
                
#     break

In [ ]:
conn.commit()

These commands are mainly selecting to see if data already exists

In [ ]:
with open('/home/ubuntu/scrapy/wiktionary_scraper/output/test3.json', 'r') as results:
    for line in results:
        this_data = json.loads(line)
        if this_data['term'] == 'cat': break
        
print('SCRAPED DATA')
print(json.dumps(this_data, indent=4))
        
word = this_data['term']
del this_data['term']

for lang, values in this_data.items():
    print()
    print('STORED DATA')
    language = lang
    
    cursor.execute(f'SELECT _id FROM etymologies e, languages l WHERE word = "{word}" and language_name = "{language}" and e.language_code = l.language_code')
    ety_id = cursor.fetchone()[0]; print('etymology:', ety_id)
    
    # Check for matching to existing entries here
    cursor.execute(f'SELECT entry_id FROM entry_connections WHERE etymology_id = {ety_id}')
    entry_ids = [item[0] for item in cursor.fetchall()]; print('entry_ids:', entry_ids)
    
    # Insert new entry ID
#     new_entry_ids
    # Add each element to SQL with that entry ID
    
    
    #What if multiple entries

    cursor.execute(f'SELECT definition FROM entry_definitions WHERE entry_id = {entry_ids[0]}')
    definitions = cursor.fetchall(); print('definitions:', definitions)

    cursor.execute(f'SELECT etymology FROM entry_etymologies WHERE entry_id= {entry_ids[0]}')
    etymologies = cursor.fetchall(); print('etymologies:', etymologies)

    cursor.execute(f'SELECT part_of_speech FROM entry_pos WHERE entry_id = {entry_ids[0]}')
    pos = cursor.fetchall(); print('pos:', pos)

    cursor.execute(f'SELECT pronunciations FROM entry_pronunciations WHERE entry_id = {entry_ids[0]}')
    pronunciations = cursor.fetchall(); print('pronunciations:', pronunciations)
    break

### Test Spider

In [1]:
%load_ext autoreload
%autoreload 2
import wiktionary_scraper.spiders.wiktionary_spider as ws
import wiktionary_scraper.pipelines as pipelines
import scrapy, mysql.connector, os, requests
from ety_utils import *
conn = mysql.connector.connect(
    user=os.environ['MYSQL_DB_USER'], 
    password=os.environ['MYSQL_DB_PASSWORD'], 
    host='127.0.0.1', 
    database='etymology_explorer_staging')
cursor = conn.cursor()

In [ ]:
# ??scrapy.http.response
# ??ws.WiktionarySpider
# ??scrapy.Request

In [5]:
cursor.execute('DELETE FROM entry_etymologies')
cursor.execute('DELETE FROM connections')
cursor.execute('DELETE FROM connection_sources')
conn.commit()

In [6]:
term='cat'#'Reconstruction:Proto-Indo-European%2Fkerp-'#'犬'
url = f'https://en.wiktionary.org/api/rest_v1/page/html/{term}'; url
temp_request = requests.get(url)
body = temp_request.content; body
status = temp_request.status_code; status
spider = ws.WiktionarySpider()
pipe = pipelines.WiktionaryScraperPipeline()
pipe.open_spider(spider)
request = scrapy.Request(url, meta = {'term': term})
response = scrapy.http.response.Response(url=url, body=body, request=request, status=status); response

epoch      trn_loss   val_loss   wd_acc     sent_acc   
    0      26.490498  22.819418  0.0        0.0       



<200 https://en.wiktionary.org/api/rest_v1/page/html/cat>

In [7]:
for resp in spider.parse(response):
#     print(resp)
    pipe.process_item(resp, spider)
pipe.close_spider(spider)

Generating connections for 1 English cat : From Middle English cat, catte, from Old English catt (“male cat”), catte (“female cat”), from Proto-Germanic *kattuz. 
Generating connections for 2 English cat : Abbreviation of catamaran.
Generating connections for 3 English cat : Abbreviation of catenate.
Generating connections for 4 English cat : Possibly a shortened form of catastrophic.
Generating connections for 5 English cat : Shortened from methcathinone.
Generating connections for 6 English cat : Shortened from catapult.
Generating connections for 7 English cat : cat (plural cats)
Generating connections for 8 English cat : Abbreviation of catfish.
Generating connections for 9 English cat : Abbreviation of caterpillar.
Generating connections for 10 Indonesian cat : From Malay cat, from Min Nan 漆 (chhat), from Middle Chinese 漆 (tsit).
Generating connections for 11 Irish cat : From Old Irish catt, from Latin cattus.
Generating connections for 12 Malay cat : From Min Nan 漆 (chhat), from 

In [ ]:
raw_connections = eval("[{'root': {'word': 'cat', 'lang': 'Middle English'}, 'desc': {'word': 'cat', 'lang': 'English'}, 'source': 1}, {'root': {'word': 'catte', 'lang': 'Middle English'}, 'desc': {'word': 'cat', 'lang': 'English'}, 'source': 1}, {'root': {'word': 'catt', 'lang': 'Old English'}, 'desc': {'word': 'cat', 'lang': 'Middle English'}, 'source': 1}, {'root': {'word': 'catte', 'lang': 'Old English'}, 'desc': {'word': 'cat', 'lang': 'Middle English'}, 'source': 1}, {'root': {'word': 'kattuz', 'lang': 'Proto-Germanic'}, 'desc': {'word': 'catt', 'lang': 'Old English'}, 'source': 1}]")
raw_connections

In [ ]:
def getOrCreateEtyId(word, language):
    sql = f'SELECT _id FROM etymologies WHERE word = {repr(word)} and language_name = {repr(language)}'
#     print(sql)
#     ety_id_result = execute_sql(sql)
    
#     if ety_id_result:
#         ety_id = ety_id_result[0][0]
#     else:
#         ety_id = self.getNewKey('_id', 'etymologies')
#         self.cursor.execute(f'INSERT INTO etymologies(_id, word, language_name) VALUES ({ety_id}, {repr(word)}, {repr(language)})')
    return np.random.randint(0,10)


for connection in raw_connections:
    
    root_id = getOrCreateEtyId(connection['root']['word'], connection['root']['lang'])
    desc_id = getOrCreateEtyId(connection['desc']['word'], connection['desc']['lang'])
    insert()
    


In [ ]:
etymology_codes = ['lang', 'key', 'None', '[0, 0]', 'None', 'lang', '[0, 0, 0]']
etymology_words = ['English', 'banana', ':', 'bananana', 'from', 'Greek', 'banananana']
make_connections(etymology_words, etymology_codes, 1)

In [ ]:
hasErrors(['lang', 'key', 'None', '[0, 0]'], ['English', 'banana', ':', 'bananana'])

### Testing

In [ ]:
cursor.execute('SELECT * FROM languages')
lc2ln = {row[1].decode(): row[0].decode() for row in cursor.fetchall()}; lc2ln['alu']

In [ ]:
cursor.execute('SELECT word, language_code FROM etymologies WHERE _id NOT IN (SELECT DISTINCT etymology_id FROM entry_connections)')
new_terms = [[row[0].decode().strip(), row[1].decode()] for row in cursor.fetchall()]

In [ ]:
url_terms = [row[0] if not row[1].endswith('-pro') else 'Reconstruction:'+lc2ln[row[1]]+'/'+row[0] for row in new_terms[:5]]; url_terms

In [ ]:
for new_term in new_terms:
    print(new_term)
    break

In [ ]:
try:
    cursor.execute("SELECT * FROM etymologies LIMIT 30")
except Exception as e:
    cursor.fetchall()
    raise(e)
    

In [ ]:
def test():
    yield 'a'
    return
    yield 'b'
    
for a in test(): print (a)